# Notebook 5
# Testing pose estimate with webcam

Loads one of the models trained, normalizes the data and predicts live the
direction looking to.


<div>
<img src="pose_landmarks_index.png" width="400"/>
</div>

In [1]:
import numpy as np
import cv2
import mediapipe as mp
import random
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pathlib import Path
import pyautogui
from joblib import dump, load

import pose_ml as pml

In [2]:
#get features
features  = pml.FEATURES
feature_desc = pml.FEATURE_DESCRIPTIONS

# get the actions
actions = pml.ACTIONS
actions_desc = pml.ACTION_DESCRIPTIONS

print("Features: ")
for key, feature in feature_desc.items():
    print(key, feature)
print("\nActions: ")
for key, action in actions_desc.items():
    print(key, action)


Features: 
(0, 2) nose, left eye
(0, 5) nose, right eye
(0, 7) nose, left ear
(0, 8) nose, right ear
(0, 9) nose, left mouth
(0, 10) nose, right mouth
(0, 11) nose, left shoulder
(0, 12) nose, right shoulder
(2, 5) left eye, right eye
(2, 7) left eye, left ear
(5, 8) right eye, right ear
(7, 8) left ear, right ear
(7, 9) left ear, left mouth
(7, 11) left ear, left shoulder
(8, 10) right ear, right mouth
(8, 12) right ear, right shoulder
(9, 10) left mouth, right mouth
(9, 11) left mouth, left shoulder
(10, 12) right mouth, right shoulder
(11, 12) left shoulder, right shoulder

Actions: 
C MIDDEN
TL LINKS BOVEN
TR RECHTS BOVEN
BL LINKS ONDER
BR RECHTS ONDER


In [3]:
project_data = Path(r"./data")
graph_path = Path(r"./graphs")
model_path = Path(r"./models")
sns.set(style="whitegrid")
color = ["Blue", "Red", "Yellow", "Green", "Grey"] 
sns.set_palette(color) 
img_size = (512, 512)

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable =False
    results =  model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [6]:
import os

model_files = [f for f in os.listdir(str(model_path)) if f.endswith('.joblib')]

# Initialize a dictionary to hold the loaded classifiers
classifiers = {}

# Loop through the model files, loading each classifier
for model_file in model_files:
    file_path = os.path.join(str(model_path), model_file)
    # Load the classifier
    clf = load(file_path)
    # Store the classifier in the dictionary with its filename as the key
    classifiers[model_file.split('.j')[0]] = clf

In [7]:
classifiers

{'DecisionTreeClassifier__': DecisionTreeClassifier(),
 'DecisionTreeClassifier__PCA': DecisionTreeClassifier(),
 'KNeighborsClassifier__': KNeighborsClassifier(),
 'KNeighborsClassifier__PCA': KNeighborsClassifier(),
 'RandomForestClassifier__': RandomForestClassifier(),
 'RandomForestClassifier__PCA': RandomForestClassifier(),
 "SVC_kernel='linear'_": SVC(kernel='linear'),
 "SVC_kernel='linear'_PCA": SVC(kernel='linear'),
 "SVC_kernel='poly'_": SVC(kernel='poly'),
 "SVC_kernel='poly'_PCA": SVC(kernel='poly'),
 'SVC__': SVC(),
 'SVC__PCA': SVC()}

## Load one of the moddels above

In [9]:
# Load one of the classifiers
# Decision tree scored best during training.
cur_model = classifiers['DecisionTreeClassifier__']
cur_model

DecisionTreeClassifier()

In [10]:
# load statistics for normalizeion
feature_stats = pd.read_json(str(model_path / 'feature_stats.json'))
# convert feature names to tuples as in the dataset
feature_stats.index = [eval(idx) if isinstance(idx, str) else idx for idx in feature_stats.index]
print(feature_stats)

              mean       std
(0, 2)    0.040256  0.010003
(0, 5)    0.039030  0.010094
(0, 7)    0.044056  0.017707
(0, 8)    0.043022  0.017324
(0, 9)    0.043643  0.008515
(0, 10)   0.044039  0.007796
(0, 11)   0.223275  0.047826
(0, 12)   0.232345  0.047433
(2, 5)    0.038287  0.007162
(2, 7)    0.030115  0.009709
(5, 8)    0.030392  0.010374
(7, 8)    0.073687  0.013152
(7, 9)    0.061654  0.017293
(7, 11)   0.220776  0.041596
(8, 10)   0.060203  0.017347
(8, 12)   0.229787  0.039111
(9, 10)   0.028796  0.005836
(9, 11)   0.180501  0.040372
(10, 12)  0.188974  0.040666
(11, 12)  0.207278  0.031399


In [11]:

# Create a VideoCapture object
#cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Loop through each frame in the video
    while cap.isOpened():
        # Read the frame
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        pml.draw_landmarks(image, results)
        # draw_custom_landmarks(image, results)
        key_points = pml.extract_keypoints(results)
        
        sample = np.array(pml.calculate_distance(key_points.reshape(len(key_points) // 4,4)))
        
        if sample.all() !=0:
            df = pd.DataFrame([sample],columns = features)
            # normalize the sample 
            df = (df - feature_stats['mean']) / feature_stats['std'] 
            
            action = cur_model.predict(df)
            act =actions_desc[ actions[action[0]]]
            pml.draw_direction(image, action[0])
            cv2.putText(image, f'KIJKT NAAR {act}', (120, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
        cv2.imshow("Camera feed:", image)
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
